In [4]:
import pandas as pd
import pymysql

# List of CSV file paths
csv_files = [
    "kerala_bus_details.csv", 
    "kadamba_bus_details.csv", 
    "west_bengal_bus_details.csv", 
    "south_bengal_bus_details.csv", 
    "chandigarh_bus_details.csv", 
    "assam_bus_details.csv",
    "north_bengal_bus_details.csv", 
    "punjab_bus_details.csv", 
    "rajasthan_bus_details.csv", 
    "uttra_pradesh_bus_details.csv"
]   

# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Add ID column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

# Clean up DataFrame
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')

# MySQL connection
myconnection = pymysql.connect(
    host="127.0.0.1", 
    user="root", 
    passwd="Narendran18*", 
    database="redbus"
)

# Drop null values
combined_df = combined_df.dropna()

# Prepare column definitions for MySQL
a = ",".join(f"{i} {j}" for i, j in zip(combined_df.columns, combined_df.dtypes)).replace("float64", "float").replace("object", "text").replace("int64", "int")

# Create table
table_name = "bus_routes"
myconnection.cursor().execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({a})")

# Insert data into MySQL
for i in range(len(combined_df)):
    myconnection.cursor().execute(f"INSERT INTO {table_name} VALUES {tuple(combined_df.iloc[i])}")
myconnection.commit()

# Close the MySQL connection
myconnection.close()
